In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import pandas.plotting as pplt

import baos_knx_parser as knx

import pymysql.cursors

import csv
import binascii
import datetime
import json

In [7]:
with open('db-config.json', 'r') as fp:
    DB_CONFIG = json.load(fp)

dbcon = pymysql.connect(
    #host='127.0.0.1',
    host=DB_CONFIG['host'],
    user=DB_CONFIG['user'],
    password=DB_CONFIG['pass'],
    db=DB_CONFIG['db'],
    #cursorclass=pymysql.cursors.DictCursor,
    cursorclass=pymysql.cursors.SSDictCursor,
)

# Verfügbare zusammenhängende Daten
-----------------------------------

* 21.Jan.-21.Feb.2017
* Jan 2014
* 11.Dez 2013-14.Jan 2014

In [8]:
DATASET_RANGES = {
    'training': (datetime.date(2017, 1, 21), datetime.date(2017, 2, 21)),
    #'validate': (datetime.date(2017, 1, 21), datetime.date(2017, 2, 21)),
    #'test': (datetime.date(2017, 1, 21), datetime.date(2017, 2, 21)),
}

def fetch_dataset(name):
    with dbcon.cursor() as cursor:
        cursor.execute("SELECT Time, Date, cemi FROM knxlog WHERE Date BETWEEN %s AND %s;",
                       [str(DATASET_RANGES[name][0]), str(DATASET_RANGES[name][1])])
        # print(str(DATASET_RANGES[name][0]), str(DATASET_RANGES[name][1]), rows)
        while True:
            row = cursor.fetchone()
            if row is None:
                break
                
            timestamp = datetime.datetime.combine(row['Date'], datetime.time(0, 0)) + row['Time']
            yield (timestamp, bytes.fromhex(row['cemi']))
            # yield knx.parse_knx_telegram(bytes.fromhex(row['cemi']), timestamp)
            
def fetch_parsed_dataset(name):
    for data in fetch_dataset(name):
        yield knx.parse_knx_telegram(data[1], data[0])

#list(fetch_dataset('training'))

In [18]:
"""
for name in DATASET_RANGES.keys():
    with open(f'knx_dataset_{name}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for data in fetch_dataset(name):
            writer.writerow((data[0], binascii.hexlify(data[1])))
"""
pass

In [19]:
with open(f'knx_dataset.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for data in fetch_dataset('training'):
        writer.writerow((data[0], binascii.hexlify(data[1])))

In [10]:
with dbcon.cursor() as cursor:
    cursor.execute("SELECT DISTINCT SourceAddress FROM knxlog")
    result = cursor.fetchall()

In [14]:
addr_set = set(map(lambda r: r['SourceAddress'], result))
addr_set

{'3.1.1',
 '3.3.29',
 '3.4.16',
 '3.4.17',
 '3.4.26',
 '3.5.10',
 '3.5.11',
 '3.5.12',
 '3.5.13',
 '3.5.14',
 '3.5.3',
 '3.5.4',
 '3.5.5',
 '3.5.54',
 '3.5.55',
 '3.5.56',
 '3.5.57',
 '3.5.58',
 '3.5.6',
 '3.5.66',
 '3.5.67',
 '3.5.69',
 '3.5.7',
 '3.5.9',
 '3.6.1',
 '3.6.10',
 '3.6.11',
 '3.6.12',
 '3.6.13',
 '3.6.14',
 '3.6.15',
 '3.6.16',
 '3.6.18',
 '3.6.19',
 '3.6.2',
 '3.6.20',
 '3.6.21',
 '3.6.22',
 '3.6.24',
 '3.6.25',
 '3.6.250',
 '3.6.27',
 '3.6.28',
 '3.6.3',
 '3.6.30',
 '3.6.31',
 '3.6.32',
 '3.6.33',
 '3.6.34',
 '3.6.35',
 '3.6.36',
 '3.6.37',
 '3.6.39',
 '3.6.4',
 '3.6.41',
 '3.6.42',
 '3.6.43',
 '3.6.44',
 '3.6.45',
 '3.6.46',
 '3.6.47',
 '3.6.5',
 '3.6.6',
 '3.6.7',
 '3.6.8',
 '3.6.9',
 '4.7.2',
 '5.1.0',
 '5.1.1',
 '6.1.0'}

In [17]:
'3.6.26' in addr_set

False

In [21]:
'3.5.18' in addr_set

False

In [ ]:
#dbcon.close()